In [30]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import environment
from collections import deque
import random
import sys

import environment
import models
import agent

import warnings
warnings.filterwarnings('ignore')

In [31]:
esn_input_size = 4
esn_reservoir_size = 1024
esn_output_size = 128
lstm_input_size = 128
lstm_num_hidden = 512
lstm_num_layers = 4
lstm_output_size = 3

In [32]:
# test on test data
esn = ESN(esn_input_size, esn_reservoir_size, esn_output_size)
esn.load_state_dict(torch.load('trained_models/esn.pt'))
lstm = LSTM(lstm_input_size, lstm_num_hidden, lstm_num_layers, lstm_output_size, esn)
lstm.load_state_dict(torch.load('trained_models/150_lstm.pt'))
agent = Agent(lstm, "crypto_data/test_data.csv")

state = agent.env.reset()
rewards = []
while True: 
    try:
        action = agent.get_action(state)
        new_state, reward, done = agent.env.step(action)
        rewards.append(reward)
        agent.remember(state, action, reward)
        agent.train_short_memory(state, action, reward)
        state = new_state
        if done:
            agent.train_long_memory()
    except StopIteration:
        break

In [33]:
print("Total profit: ", agent.env.profit)

print("Percentage profit: ", agent.env.profit / 46445 * 100)

Total profit:  tensor(42560.6992)
Percentage profit:  tensor(91.6368)


In [34]:
# no of 1 in rewards
print("No of 1 in rewards: ", rewards.count(1))
print("No of -1 in rewards: ", rewards.count(-1))
print("No of 0 in rewards: ", rewards.count(0))

# in percentage
print("Percentage of 1 in rewards: ", round(rewards.count(1)/len(rewards)*100, 1))
print("Percentage of -1 in rewards: ", round(rewards.count(-1)/len(rewards)*100, 1))
print("Percentage of 0 in rewards: ", round(rewards.count(0)/len(rewards)*100, 1))

No of 1 in rewards:  550
No of -1 in rewards:  163
No of 0 in rewards:  6
Percentage of 1 in rewards:  76.5
Percentage of -1 in rewards:  22.7
Percentage of 0 in rewards:  0.8
